In [1]:
# Import necessary libraries
import os
import cv2
import random
import shutil
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Ignore warnings
warnings.filterwarnings("ignore", category=UserWarning, module="urllib3")

# Define base paths
base_folder = "/Users/kisanetgabreselassie/Downloads/archive 2"
output_folder = "/Users/kisanetgabreselassie/Downloads/archive_2_downsized"

# Define splits and categories
sets = ['train', 'test', 'valid']
categories = ['wildfire', 'nowildfire']

# Ratio for downsizing
downsizing_factor = 0.5

# Create output directories
for dataset in sets:
    for category in categories:
        new_dir = os.path.join(output_folder, dataset, category)
        os.makedirs(new_dir, exist_ok=True)

# Function to downsize a dataset
def downsize_dataset():
    for dataset in sets:
        for category in categories:
            # Define paths
            category_path = os.path.join(base_folder, dataset, category)
            new_category_path = os.path.join(output_folder, dataset, category)

            # Get list of image files
            image_files = os.listdir(category_path)
            
            # Calculate the number of images to keep
            num_images_to_keep = int(len(image_files) * downsizing_factor)

            # Randomly sample the images
            sampled_images = random.sample(image_files, num_images_to_keep)

            # Copy the sampled images to the new downsized dataset
            for image_file in sampled_images:
                old_image_path = os.path.join(category_path, image_file)
                new_image_path = os.path.join(new_category_path, image_file)
                shutil.copy(old_image_path, new_image_path)
    
    print("Dataset downsized successfully.")


/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
downsize_dataset()

# Update paths to point to the downsized dataset
train_dir = os.path.join(output_folder, 'train')
valid_dir = os.path.join(output_folder, 'valid')
test_dir = os.path.join(output_folder, 'test')


# from PIL import ImageFile
#ImageFile.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from train and validation directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (wildfire vs nowildfire)
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
print("Data Generators ready for training.")



Dataset downsized successfully.
Found 29991 images belonging to 2 classes.
Found 6178 images belonging to 2 classes.
Data Generators ready for training.


In [3]:
# Build the CNN model
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),  # Explicit Input layer
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])


# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,  # Adjust the number of epochs as needed
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

# Save the trained model
model.save('wildfire_prediction_model.h5')
print("hello")


/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
618/938 ━━━━━━━━━━━━━━━━━━━━ 3:36 677ms/step - accuracy: 0.8454 - loss: 0.3736

2024-11-16 12:18:37.572385: W tensorflow/core/framework/op_kernel.cc:1829] UNKNOWN: OSError: image file is truncated (16 bytes not processed)
Traceback (most recent call last):

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 247, in _finite_generator
    yield self.py_dataset[i]

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/py

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
OSError: image file is truncated (16 bytes not processed)
Traceback (most recent call last):

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/kisanetgabreselassie/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py", line 247, in _finite_generator
    yield self.py_dataset[i]

  File "/Users/kisan

In [ ]:
loss, accuracy = model.evaluate(valid_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


In [ ]:
print("hello")